## Analyze rotation of water with pyscf calculation

In [ ]:
from pyscf import gto
import numpy as np
from e3nn import o3
from collections import defaultdict
import pandas as pd
from pymatgen.core.structure import Molecule
from pyscf import dft
import plotly.express as px
from pprint import pprint
from minimal_basis.transforms.rotations import RotationMatrix
from instance_mongodb import instance_mongodb_sei

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_water_initial_structures

data = defaultdict(list)

for doc in collection.find():
    pymatgen_molecule = doc["molecule"]
    pymatgen_molecule = Molecule.from_dict(pymatgen_molecule)
    coordinates = np.array(pymatgen_molecule.cart_coords)
    species = [str(site.specie) for site in pymatgen_molecule]
    mol = gto.Mole()

    atom_string = ''.join([f"{species[i]} {coordinates[i][0]} {coordinates[i][1]} {coordinates[i][2]}; " for i in range(len(species))])
    atom_string = atom_string[:-2]

    mol.atom = atom_string
    mol.basis = 'def2-svp'
    mol.build()

    rks_h2o = dft.RKS(mol)
    rks_h2o.xc = 'b3lyp'

    rks_h2o.kernel()

    data["molecule"].append(mol)
    data["rks"].append(rks_h2o)
    data["euler_angles"].append(doc["euler_angles"])


In [ ]:
coeff_matrices = []
for idx, molecule in enumerate(data['molecule']):
    rks = data['rks'][idx]
    mo_coeff = rks.mo_coeff
    coeff_matrices.append(mo_coeff)

    if idx == 0:
        labels = molecule.ao_labels()
        eigenvalues = rks.mo_energy

coeff_matrices = np.array(coeff_matrices)
fig = px.imshow(coeff_matrices, animation_frame=0)
fig.update_xaxes(title_text='MO Index')
fig.update_yaxes(title_text='AO Index')

fig.update_yaxes(ticktext=labels, tickvals=np.arange(len(labels)))
# Add the eigenvalues of the MO as the x-axis ticktext
fig.update_xaxes(ticktext=np.round(eigenvalues, 2), tickvals=np.arange(len(eigenvalues)))

fig.show()

In [ ]:
def labels_to_dataframe(pyscf_labels):
    """Convert the pyscf labels to irreps."""
    pyscf_labels = [label.split()[0:3] for label in pyscf_labels]
    pyscf_labels = [
        d[:-1] + [d[-1][0]] + [d[-1][1]] + [d[-1][2:]] for d in pyscf_labels
    ]
    pyscf_labels = pd.DataFrame(
        pyscf_labels, columns=["atom_idx", "atom_name", "n", "l", "m"]
    )
    pyscf_labels["atom_idx"] = pyscf_labels["atom_idx"].astype(int)
    pyscf_labels["n"] = pyscf_labels["n"].astype(int)
    return pyscf_labels

df_labels = labels_to_dataframe(labels)
l = df_labels.l.values
m = df_labels.m.values

idx = 0
irreps = ""
while idx < len(l):
    if l[idx] == "s":
        irreps += "+1x0e"
        idx += 1
    elif l[idx] == "p":
        irreps += "+1x1o"
        idx += 3
    elif l[idx] == "d":
        irreps += "+1x2e"
        idx += 5
irreps = irreps[1:]
print(f"irreps: {irreps}")
